Word Level LSTM for five-grams (sequence length: 4)

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [16]:
ls

data.txt  model.h5  republic.txt  tokenizer.pkl  Word_Level_LSTM_Config_3.ipynb


In [7]:
cd ../gdrive/My\ Drive/NLP/Project

[Errno 2] No such file or directory: 'gdrive/My Drive/NLP/Project/Final Project/LSTM/'
/


In [0]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load
in_filename = 'republic.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1


In [23]:
sequences[1:10]

[[],
 [30, 1614, 5, 26, 1, 163, 2, 2549, 2550, 35, 44, 2871, 3, 28],
 [568, 44, 3940, 4907, 21, 37, 1062, 73, 235, 73, 248, 13],
 [2551, 163, 73, 157, 1, 379, 2, 1, 286, 264, 1178, 1325],
 [28, 30, 1614, 13, 3323, 35, 2552, 264, 1404],
 [],
 [],
 [2069, 1, 227],
 []]

In [0]:
flat_list = [item for sublist in sequences for item in sublist]

In [0]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 4
dataX = []
dataY = []
for i in range(0, len(flat_list) - seq_length, 1):
	seq_in = flat_list[i:i + seq_length]
	seq_out = flat_list[i + seq_length]
	dataX.append(seq_in)
	dataY.append(seq_out)
n_patterns = len(dataX)

In [26]:
n_patterns

221273

In [0]:
dataX = array(dataX)
dataY = array(dataY)

In [28]:
dataX.shape, dataY.shape

((221273, 4), (221273,))

In [29]:
# separate into input and output
X, y = dataX, dataY
y = to_categorical(y, num_classes=vocab_size)
seq_length = dataX.shape[1]

# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=50)

# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 50)             551800    
_________________________________________________________________
lstm_1 (LSTM)                (None, 4, 100)            60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 11036)             1114636   
Total params: 1,817,336
Trainable params: 1,817,336
Non-trainable params: 0
_________________________________________________________________
None


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epo

In [17]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

# load the model
model = load_model('model.h5')
# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))

Using TensorFlow backend.













Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [0]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

In [36]:
# select a seed text
seed_text = "very true he"
# generate new text
generated = generate_seq(model, tokenizer, 4, seed_text, 1)
print(generated)

said


In [35]:
# select a seed text
seed_text = "very true"
# generate new text
generated = generate_seq(model, tokenizer, 4, seed_text, 2)
print(generated)

he said


In [19]:
# for our verification
import nltk
from nltk.util import ngrams
import collections

blogs = open('republic.txt', 'rt')
text = blogs.read()
blogs.close()
corpus = text
# We will remove double quotes, !, ? $, #, etc
# specify to translate chars 
str1 = ""
# specify to replace with 
str2 = ""
# delete chars 
str3 = "\"!?#$%^&*+"

trg = corpus
table = trg.maketrans(str1, str2, str3)
corpus = trg.translate(table)
words = corpus.split()

freq_dist = nltk.FreqDist(words)
freq_dist.most_common(10) # the most common words in our corpus.

# esBigrams = ngrams(words, 2)
# esBigramFreq = collections.Counter(esBigrams)
# esBigramFreq.most_common(10)

# esTrigrams = ngrams(words, 3)
# esTrigramFreq = collections.Counter(esTrigrams)
# esTrigramFreq.most_common(10)

esQuadgrams = ngrams(words, 4)
esQuadgramFreq = collections.Counter(esQuadgrams)
esQuadgramFreq.most_common(10)

[(('for', 'the', 'sake', 'of'), 28),
 (('Very', 'true,', 'he', 'said.'), 26),
 (('the', 'interest', 'of', 'the'), 25),
 (('What', 'do', 'you', 'mean'), 24),
 (('at', 'the', 'same', 'time'), 22),
 (('the', 'rest', 'of', 'the'), 21),
 (('Yes,', 'he', 'said,', 'that'), 21),
 (('in', 'the', 'case', 'of'), 20),
 (('as', 'well', 'as', 'of'), 18),
 (('the', 'idea', 'of', 'good'), 18)]